Builds the frame-level dataset TAD-YOLO-CLS/ from clips_index.json (one frame per clip), creates tad_cls.yaml, and fine-tunes YOLOv8n-cls on train/val; also runs the built-in Ultralytics test evaluation.

In [1]:

import os
import json

PROJECT_ROOT = "/home/olzhas/programming/traffic-accident-edge"
DATA_ROOT = os.path.join(PROJECT_ROOT, "TAD-benchmark")

SPLITS_PATH = os.path.join(DATA_ROOT, "splits.json")
CLIPS_INDEX_PATH = os.path.join(DATA_ROOT, "clips_index.json")

print("DATA_ROOT exists:", os.path.exists(DATA_ROOT))
print("SPLITS_PATH exists:", os.path.exists(SPLITS_PATH))
print("CLIPS_INDEX_PATH exists:", os.path.exists(CLIPS_INDEX_PATH))

with open(SPLITS_PATH, "r") as f:
    splits = json.load(f)

with open(CLIPS_INDEX_PATH, "r") as f:
    clips = json.load(f)

print("acc_train videos:", len(splits["acc_train"]))
print("norm_train videos:", len(splits["norm_train"]))
print("acc_val videos:", len(splits["acc_val"]))
print("norm_val videos:", len(splits["norm_val"]))
print("acc_test videos:", len(splits["acc_test"]))
print("norm_test videos:", len(splits["norm_test"]))
print("Total clips:", len(clips))
print("Sample clip:", clips[0])

DATA_ROOT exists: True
SPLITS_PATH exists: True
CLIPS_INDEX_PATH exists: True
acc_train videos: 208
norm_train videos: 88
acc_val videos: 53
norm_val videos: 23
acc_test videos: 16
norm_test videos: 16
Total clips: 22037
Sample clip: {'video_path': '/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/train/accident_1/videox3_10.mp4', 'frame_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'label': 1, 'split': 'train'}


In [2]:
import os
import shutil

YOLO_DATA_ROOT = os.path.join(DATA_ROOT, "TAD-YOLO-CLS")

for split in ["train", "val", "test"]:
    for cls in ["accident", "normal"]:
        out_dir = os.path.join(YOLO_DATA_ROOT, split, cls)
        if os.path.exists(out_dir):
            shutil.rmtree(out_dir)
        os.makedirs(out_dir, exist_ok=True)

print("YOLO_DATA_ROOT:", YOLO_DATA_ROOT)
for split in ["train", "val", "test"]:
    for cls in ["accident", "normal"]:
        path = os.path.join(YOLO_DATA_ROOT, split, cls)
        print(path, "exists:", os.path.exists(path))

YOLO_DATA_ROOT: /home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS
/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/train/accident exists: True
/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/train/normal exists: True
/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/val/accident exists: True
/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/val/normal exists: True
/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/test/accident exists: True
/home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/test/normal exists: True


In [3]:
import cv2
from tqdm import tqdm


def save_frame(video_path, frame_idx, out_path):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    if not ret or frame is None:
        return False
    cv2.imwrite(out_path, frame)
    return True


def label_to_name(label):
    return "accident" if label == 1 else "normal"


num_saved = 0
num_failed = 0

for clip in tqdm(clips, desc="Extracting frames"):
    video_path = clip["video_path"]
    frame_indices = clip["frame_indices"]
    label = clip["label"]
    split = clip["split"]

    center_idx = frame_indices[len(frame_indices) // 2]

    cls_name = label_to_name(label)
    video_base = os.path.splitext(os.path.basename(video_path))[0]
    out_name = f"{video_base}_f{center_idx:05d}.jpg"

    out_dir = os.path.join(YOLO_DATA_ROOT, split, cls_name)
    out_path = os.path.join(out_dir, out_name)

    if os.path.exists(out_path):
        continue

    ok = save_frame(video_path, center_idx, out_path)
    if ok:
        num_saved += 1
    else:
        num_failed += 1

print("Saved images:", num_saved)
print("Failed extractions:", num_failed)

Extracting frames: 100%|██████████| 22037/22037 [42:27<00:00,  8.65it/s] 

Saved images: 21983
Failed extractions: 54


In [4]:
import glob

for split in ["train", "val", "test"]:
    for cls in ["accident", "normal"]:
        pattern = os.path.join(YOLO_DATA_ROOT, split, cls, "*.jpg")
        files = glob.glob(pattern)
        print(f"{split}/{cls}: {len(files)} images")

data_yaml = os.path.join(YOLO_DATA_ROOT, "tad_cls.yaml")

with open(data_yaml, "w") as f:
    f.write(f"path: {YOLO_DATA_ROOT}\n")
    f.write("train: train\n")
    f.write("val: val\n")
    f.write("test: test\n")
    f.write("names:\n")
    f.write("  0: accident\n")
    f.write("  1: normal\n")

print("\nWrote YOLO config to:", data_yaml)

train/accident: 12071 images
train/normal: 3949 images
val/accident: 2959 images
val/normal: 944 images
test/accident: 991 images
test/normal: 1069 images

Wrote YOLO config to: /home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/tad_cls.yaml


In [ ]:
from ultralytics import YOLO
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

model = YOLO("yolov8n-cls.pt")

results = model.train(
    data=YOLO_DATA_ROOT,
    epochs=30,
    imgsz=224,
    batch=64,
    device=0
)

print("Training finished. Results saved to:", results.save_dir)

CUDA available: True
GPU: NVIDIA GeForce RTX 2080 Ti
New https://pypi.org/project/ultralytics/8.3.230 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.13.9 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 10799MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/olzhas/Desktop/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train

In [5]:
from ultralytics import YOLO

best_weights = os.path.join(
    PROJECT_ROOT,
    "notebooks",
    "runs",
    "classify",
    "train2",
    "weights",
    "best.pt",
)

print("Using weights:", best_weights)

model = YOLO(best_weights)

val_metrics = model.val(data=data_yaml, split="val", device=0)
print("VAL metrics:", val_metrics.results_dict)

test_metrics = model.val(data=data_yaml, split="test", device=0)
print("TEST metrics:", test_metrics.results_dict)

Using weights: /home/olzhas/programming/traffic-accident-edge/notebooks/runs/classify/train2/weights/best.pt
Ultralytics 8.3.229 🚀 Python-3.13.9 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 10799MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2856.0±674.7 MB/s, size: 68.7 KB)
val: Scanning /home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/val... 3903 images, 0 corrupt: 100% ━━━━━━━━━━━━ 3903/3903 1.9Kit/s 2.1s0.1s
val: New cache created: /home/olzhas/programming/traffic-accident-edge/TAD-benchmark/TAD-YOLO-CLS/val.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 244/244 29.7it/s 8.2s0.1s
                   all      0.963          1
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/olzhas/programming/traffic-accident-edge/notebooks/runs/classify/val2
VAL metrics: {'metrics/accuracy_top1':